# 🚀 BDA Contest - SOTA Solution with AutoGluon

## 핵심 전략: AutoGluon (AutoML)
- **최신 논문 기술 집약체**: Neural Network + XGBoost + LightGBM + CatBoost 자동 앙상블
- **Deep Learning**: 정형 데이터 전용 신경망(Tabular Neural Network) 자동 적용
- **NLP 통합**: 텍스트 컬럼(지원동기 등)을 NLP 모델로 임베딩하여 활용
- **Stacking & Bagging**: 수십 개 모델을 겹쳐 쌓아 과적합 방지 및 성능 극대화
- **Threshold Calibration**: 데이터 불균형 해결을 위한 분위수(Quantile) 기반 Cut-off

In [ ]:
# 1. AutoGluon 설치 (필수)
# 주의: 설치 후 런타임 재시작이 필요할 수 있습니다. (RESTART SESSION 버튼 클릭)
!pip install autogluon -q

import pandas as pd
import numpy as np
import re
from autogluon.tabular import TabularPredictor

print("Setup Complete! If you see errors, please Restart Session.")

In [ ]:
# 2. 데이터 로드
RAW_TRAIN_URL = 'https://raw.githubusercontent.com/choicompany/bdadacon/refs/heads/main/rawdata/train.csv'
RAW_TEST_URL = 'https://raw.githubusercontent.com/choicompany/bdadacon/refs/heads/main/rawdata/test.csv'

train = pd.read_csv(RAW_TRAIN_URL)
test = pd.read_csv(RAW_TEST_URL)

print(f"Train Shape: {train.shape}, Test Shape: {test.shape}")

In [ ]:
# 3. 최소한의 피처 엔지니어링 (텍스트는 원본 유지 -> AutoGluon이 읽음)
# AutoGluon은 원본 데이터를 잘 다루므로, 과도한 전처리는 오히려 독이 될 수 있음.
# 하지만 '개수' 정보는 명시적으로 주는 것이 좋음.

def count_items(text):
    if pd.isna(text) or str(text).strip() == '': return 0
    return str(text).count(',') + 1

# 리스트형 데이터 개수 피처 추가
multi_select_cols = [
    'certificate_acquisition', 'desired_certificate', 
    'desired_job', 'desired_job_except_data',
    'onedayclass_topic', 'expected_domain'
]

for df in [train, test]:
    # 1. 개수 피처 생성
    for col in multi_select_cols:
        if col in df.columns:
            df[f'{col}_count'] = df[col].apply(count_items)
    
    # 2. 전공 데이터 특수 처리 (중요)
    # False/True 문자열을 0/1로 변환
    if 'major_data' in df.columns:
        df['major_data'] = df['major_data'].astype(str).str.lower().map({'true': 1, 'false': 0}).fillna(0).astype(int)
    
    # 3. 불필요 컬럼 ID 제거 (학습에서 제외)
    # generation은 의미 없어 보이나 AutoGluon이 알아서 판단하게 둠

# AutoGluon은 Feature Engineering을 내부적으로 수행하므로 여기서 멈춤.
print("Feature Engineering Done.")

In [ ]:
# 4. AutoGluon 학습 (Highest Quality)
# presets='best_quality' : Bagging, Stacking, Neural Network 등 모든 기술 총동원
# time_limit : 학습 시간 제한 (초 단위). 600초(10분) ~ 1200초(20분) 권장

metric = 'f1'  # F1 Score 최적화
label = 'completed'  # 타겟 컬럼
save_path = 'ag_models_best'  # 모델 저장 경로

predictor = TabularPredictor(
    label=label,
    eval_metric=metric,
    path=save_path
).fit(
    train,
    presets='best_quality',   # 최강 성능 모드
    time_limit=60*10,         # 10분 동안 학습
    ag_args_fit={'num_gpus': 1} # GPU 사용
)

In [ ]:
# 5. 성능 평가 (Leaderboard)
# 어떤 모델들이 만들어졌고 성능이 어떤지 확인
predictor.leaderboard(train, silent=True)

In [ ]:
# 6. 예측 및 제출 파일 생성 (Threshold 자동 보정)

# 중요: ID 컬럼을 제외한 데이터만 예측에 사용
test_data = test.drop(columns=['ID'])

# 확률값 예측 (Label 1일 확률)
pred_probs = predictor.predict_proba(test_data)
# AutoGluon 버전에 따라 컬럼명이 다르므로 확인 (usually 1 or '1')
prob_1 = pred_probs.iloc[:, 1]

print("예측 확률값 통계:")
print(prob_1.describe())

# --- [핵심] Threshold 조정 ---
# 학습 데이터의 Target 비율이 약 30%였습니다.
# 따라서 확률값 상위 30% 정도를 1로 예측하는 것이 비율상 맞습니다.
# 0.5로 고정하면 모든 예측값이 0이 되거나 1이 되는 참사가 발생할 수 있습니다.

target_ratio = 0.31  # 목표 합격률 (Train 데이터 기준 약간 여유 있게)
quantile_th = 1 - target_ratio
threshold = prob_1.quantile(quantile_th)

print(f"\n조정된 Threshold (Top {target_ratio*100}%): {threshold:.4f}")

final_preds = (prob_1 >= threshold).astype(int)

print(f"최종 예측 분포: 0={sum(final_preds==0)}, 1={sum(final_preds==1)}")

submission = pd.DataFrame({
    'ID': test['ID'],
    'completed': final_preds
})

submission.to_csv('submission.csv', index=False)
print("Saved: submission.csv")
print(submission.head())

In [ ]:
# 7. 다운로드
from google.colab import files
try:
    files.download('submission.csv')
except:
    print("다운로드가 자동으로 시작되지 않으면 왼쪽 파일 탭에서 submission.csv를 우클릭하여 다운로드하세요.")